# Lab 3 - Forenzika USB uređaja
### Zadatak 1
Pronaći sve USB-ove koji su se prvi put spojili na računalo i vrijeme kad su se spojili na računalo.

In [ ]:
import requests
def download_file_with_custom_headers(url, local_filename):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }
    
    with requests.get(url, headers=headers, stream=True) as r:
        r.raise_for_status()  # This will throw an exception for non-200 responses
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                f.write(chunk)
    print("File downloaded successfully.")


# Example usage
file_url = 'https://fesb-my.sharepoint.com/:u:/g/personal/toperkov_fesb_hr/ERP3tpm9FRRIkk82lCHbQpIBGu-9efbxohQv6dZ6g2B2AQ?download=1'
output_dir = './setupapi.dev2.log'
download_file_with_custom_headers(file_url, output_dir)

### 2. Extract data from log file

In [3]:
import re
import datetime
import pandas as pd
import numpy as np

usb_devices = []


log_path= "setupapi.dev2.log"

# Read the contents of the setupapi.dev.log file
with open(log_path, "r") as log_file:
     # Store information about each USB device in a dictionary
     for line in log_file:
          match = re.match(r'^>>>  \[Device Install.*#(Disk&Ven_[A-Za-z0-9]+)&(Prod_([\w\s\S]+?))&(Rev_([\w\s\S]+?))#([\w\s\S]+?)#.*\]', line)
          if (not match):
               continue
     
     # Find all USB device installation events and extract information about each device
          usb_devices.append({
               "devicer_vendor_id": match.group(1),
               "device_product_id": match.group(2),
               "device_instance_id": match.group(4),
               "event_time": next(log_file).split("start")[1].split("\n")[0],
               "serial_number": match.group(6)
          })

df = pd.DataFrame(usb_devices)
df.tail(20)

,devicer_vendor_id,device_product_id,device_instance_id,event_time,serial_number
0,Disk&Ven_Verbatim,Prod_STORE_N_GO,Rev_1100,2022/11/03 15:00:17.389,1311130000000480&0
1,Disk&Ven_SMI,Prod_USB_DISK,Rev_1100,2023/02/01 11:10:46.945,6&7d7590d&0
2,Disk&Ven_VendorCo,Prod_ProductCode,Rev_2.00,2023/02/06 12:50:42.455,920707402A3F8066956&0
3,Disk&Ven_VendorCo,Prod_ProductCode,Rev_2.00,2023/02/06 12:51:00.660,9207113E5CF83172266&0
4,Disk&Ven_Kingston,Prod_DataTraveler_2.0,Rev_PMAP,2023/02/06 12:52:25.253,AC220B280549EF6059EEBAFE&0
5,Disk&Ven_Kingston,Prod_DataTraveler_2.0,Rev_PMAP,2023/03/23 08:13:57.618,AC220B280549EF6059EEBAFE&0
6,Disk&Ven_JetFlash,Prod_TS128MJF2A,Rev_1.00,2023/03/29 16:28:31.704,6&22ff249f&1
7,Disk&Ven_JetFlash,Prod_TS128MJF2A,Rev_1.00,2023/03/29 16:32:50.762,6&104573fa&1


In [6]:
import os, time
import pandas as pd
import hashlib
import mimetypes

dir_path = 'E:/'

# create an empty list to store the file names
file_names = []
extensions = []
md5s=[]
sha1s=[]
sha256s=[]
extension_matches=[]
creation_times=[] 
modification_times=[]
access_times=[]

def file2hash(filename, hash_function):
    with open(filename, 'rb', buffering = 0) as f:
        return hash_function(f.read()).hexdigest()
    
for file in os.listdir(dir_path):
    path = os.path.join(dir_path, file)
    if os.path.isfile(path):
        name, extension = os.path.splitext(file)
        file_names.append(file)
        extensions.append(extension)
        md5s.append(file2hash(path,hashlib.md5))
        sha1s.append(file2hash(path, hashlib.sha1))
        sha256s.append(file2hash(path, hashlib.sha256))
        creation_times.append(time.ctime(os.path.getctime(dir_path+file)))
        modification_times.append(time.ctime(os.path.getmtime(dir_path+file)))
        access_times.append(time.ctime(os.path.getatime(dir_path+file)))

df2 = pd.DataFrame({
    'file_name': file_names,
    'extension': extensions,
    'md5': md5s,
    'sha1': sha1s,
    'sha256': sha256s,
    'creation_times': creation_times,
    'modification_times': modification_times,
    'access_times': access_times
})

print(df2)



            file_name extension                               md5  \
0  Secret_file_11.txt      .txt  187b0341b64a0e16d74ea3b50e22e6c2   
1  Secret_file_92.pdf      .pdf  4266ea20320c598bc5d7b1d731e3e9e9   

                                       sha1  \
0  08152ff79a16f2a181174eb5fe8de0197f4d722f   
1  1a30a1d139131e7105a1896d7719c85c03003a59   

                                              sha256  \
0  5f90ecf090772e3366353171a2e8b9c4001934cdc3a5e0...   
1  3e174794698cbdf84d9a5cd352f755d0e6bdfc5dc65e0d...   

             creation_times        modification_times  \
0  Wed Mar 29 17:35:08 2023  Mon Oct 28 16:08:46 2019   
1  Wed Mar 29 17:35:10 2023  Mon Oct 28 16:09:32 2019   

               access_times  
0  Wed Mar 29 01:00:00 2023  
1  Wed Mar 29 01:00:00 2023  
